In [1]:

import numpy as np
import pandas as pd
import math
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
import datetime
from sklearn.model_selection import GridSearchCV

In [2]:
test = pd.read_csv("../input/tp2grupo3/test.csv")
train = pd.read_csv("../input/tp2grupo3/train.csv")

train.head

<bound method NDFrame.head of             id                                             titulo  \
0       254099                                  depto. tipo a-402   
1        53461                     condominio horizontal en venta   
2       247984              casa en venta urbi 3 recamaras tonala   
3       209067  casa sola en toluca zinacantepec con credito i...   
4       185997                                     paseos del sol   
...        ...                                                ...   
239995  119879  bonita casas de 2 recamaras a 10 minutos del c...   
239996  259178  casa en condominio a 10 min. del centro de toluca   
239997  131932                                   nicolas san juan   
239998  146867                      casa sola. javier rojo gomez.   
239999  121958  departamento en bosques de las lomas  / av. st...   

                                              descripcion     tipodepropiedad  \
0       depto. interior de 80.15m2, consta de sala com...   

In [3]:
test['fecha'] = pd.to_datetime(test['fecha'])
test['year'] = test['fecha'].dt.year
test['month']= test['fecha'].dt.month

train['fecha'] = pd.to_datetime(train['fecha'])
train['year'] = train['fecha'].dt.year
train['month']= train['fecha'].dt.month

In [4]:
train = train.drop(['fecha','titulo','descripcion','direccion','lat','lng'], axis=1)
test = test.drop(['fecha','titulo','descripcion','direccion','lat','lng'], axis=1)
train = train.fillna(value = {'provincia': 0, 'tipodepropiedad': 0, 'ciudad':0})
test = test.fillna(value = {'provincia': 0, 'tipodepropiedad': 0, 'ciudad':0})

ciudades_en_train = train.ciudad.unique()
ciudades_en_test = test.ciudad.unique()

tipodepropiedad_en_train = train.tipodepropiedad.unique()
tipodepropiedad_en_test = test.tipodepropiedad.unique()

provincia_en_train = train.provincia.unique()
provincia_en_test = test.provincia.unique()

In [5]:
train = pd.get_dummies(train, dummy_na=True)
test = pd.get_dummies(test, drop_first=True)

In [6]:
X = train.drop("precio", axis=1)
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

In [7]:
xgb1 = XGBRegressor(tree_method = 'gpu_hist', gpu_id = 0)

#Ir cambiando parametros para ver como functiona

parameters = {'nthread':[4], 
              'objective':['reg:linear'],
              'learning_rate': [0.04], 
              'max_depth': [11], 
              'min_child_weight': [9], 
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7], 
              'n_estimators': [1500],  
            'gamma': [0]} 

#0.7876343276013036 con 700 estimadores y 9 depth con fechas parseadas
#0.7903438420940652 con 700 estimadores y 9 depth
#0.7919269007179323 con 600 estimadores y 11 depth
#0.8022990816922799 con 700 estimadores , 9 depth, 5 min_child_weight
#0.8037024620856591 con 600 estimadores , 11 depth, 5 min_child_weight
#0.8037805503151154 con 600 estimadores, 11 depth, 9 min_child_weight
#0.805378985255433 con 1000 estimadores, 11 depth, 9 min_child_weight
#0.8056895246147905 con 1500 estimadores, 11 depth, 9 min_child_weight

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,y_train,verbose=True)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:  4.0min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:  4.0min finished
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.8056895246147905
{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.04, 'max_depth': 11, 'min_child_weight': 9, 'n_estimators': 1500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [8]:
preds2 = xgb_grid.predict(X_test)


In [9]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds2)

532239.5212166341

In [10]:
for ciudad in ciudades_en_train:
    if ciudad not in ciudades_en_test:
        test['ciudad_'+ str(ciudad)] = 0
        
for provincia in provincia_en_train:
    if provincia not in provincia_en_test:
        test['provincia_'+ str(provincia)] = 0
        
for tipodepropiedad in tipodepropiedad_en_train:
    if tipodepropiedad not in tipodepropiedad_en_test:
        test['tipodepropiedad_'+ str(tipodepropiedad)] = 0

In [11]:
some_list = ['tipodepropiedad_Apartamento', 'ciudad_Abasolo', 'provincia_Aguascalientes', 'tipodepropiedad_nan','provincia_nan', 'provincia_0', 'tipodepropiedad_0', 'ciudad_nan', 'ciudad_0']
for item in some_list:
    test[item] = 0

In [12]:
test = test[list(X.columns.values)]
preds = xgb_grid.predict(test)

In [13]:
res = pd.DataFrame(preds, index=test.index, columns=['precio'])

In [14]:
res2 = res
res2['id'] = test['id']
res2 = res.reset_index(drop=True)
res2.head()
res2.set_index('id', inplace = True)
#res.columns = ['id', 'target']
#res.set_index('id', inplace = True)
res2.rename(columns={'precio':'target'}, inplace=True)
res2.head()


#res2.to_csv("/kaggle/working/submission_xgboost_city_4.csv", header=True)

,target
id,
4941,6.870004e+06
51775,9.920920e+05
115253,2.411636e+06
299321,1.290262e+06
173570,6.342486e+05


In [ ]:
res2.to_csv("/kaggle/working/submission_xgboost_city_8.csv", header=True)